In [105]:
# 1. 필요 라이브러리 추가 
import pandas as pd
import numpy as np
from tqdm import tqdm

### 📌 절차 

✅ 1. 형태소 분석 - KoNLPy 활용 → 한국어 분류하기   
✅ 2. 형태소 분석 - NLTK 활용 → 영어 (Alpha) 분류하기   
✅ 3. 데이터 merge(1번, 2번)   
✅ 4. 한국어 문장 배열 요소의 구성 순위 규칙 찾아서 만들기   
✅ 5. 규칙에 맞는 품사를 랜덤으로 3번 데이터에서 불러와서 1개 문장씩 생성하기 → 1000개 생성   

-----
✅ 6. 생성된 문장 배열을 KOTE 활용하여 감성 분류 라벨링 진행 → 가장 높은 점수   
✅ 7. 3개의 테마에서 정해둔 기준에 맞는 감성의 문장만 끌어오기   

### 한국어 어순 

1. ‘주어-(목적어)-서술어［S-(O)-V］’   
2. 수식어(관형어, 부사어)는 항상 피수식어(체언, 용언) 앞에   
3. 이에 부수하여 간접적 관계에 있는 성분들(관형어는 제외)은 비교적 자유로운 어순    

In [106]:
from konlpy.tag import Okt, Kkma

text = "그대 떠난 이 거리에 남아 있는 바람소리"

# Okt 분석
okt = Okt()
print("Okt 형태소:", okt.morphs(text))
print("Okt 품사 태깅:", okt.pos(text))

# Kkma 분석
kkma = Kkma()
print("Kkma 형태소:", kkma.morphs(text))
print("Kkma 품사 태깅:", kkma.pos(text))

Okt 형태소: ['그대', '떠난', '이', '거리', '에', '남아', '있는', '바람소리']
Okt 품사 태깅: [('그대', 'Noun'), ('떠난', 'Verb'), ('이', 'Noun'), ('거리', 'Noun'), ('에', 'Josa'), ('남아', 'Noun'), ('있는', 'Adjective'), ('바람소리', 'Noun')]
Kkma 형태소: ['그대', '떠나', 'ㄴ', '이', '거리', '에', '남', '아', '있', '는', '바람', '소리']
Kkma 품사 태깅: [('그대', 'NP'), ('떠나', 'VV'), ('ㄴ', 'ETD'), ('이', 'MDT'), ('거리', 'NNG'), ('에', 'JKM'), ('남', 'VV'), ('아', 'ECD'), ('있', 'VXV'), ('는', 'ETD'), ('바람', 'NNG'), ('소리', 'NNG')]


### 데이터 불러오기 

In [107]:
# 데이터 불러오기
# data = pd.read_csv("../data/datamelon.csv")
data = pd.read_csv("../data/melon_indi_인디음악_2025-01-22_14-29-01.csv")

In [108]:
data.columns

Index(['title', 'artist', 'genre', 'release_date', 'like_cnt', 'lyrics'], dtype='object')

In [109]:
# data['lyrics'] = data['lyrics'].str.replace(r"[^\w\s\n]", "", regex=True)
data['lyrics'] = data['lyrics'].str.replace(r"[^\w\s]", "", regex=True)

In [110]:
# 'title'과 'artist' 컬럼을 기준으로 중복 제거 : SONG ID 를 지표로 하면 좋은데 나는 바보 ㅠㅠ -> 아 강사님꺼 써야겠다 ^0^ 
df_unique = data.drop_duplicates(subset=['title', 'artist'])
len(df_unique)

50

In [111]:
lyrics = df_unique['lyrics'].tolist()
print(type(lyrics), len(lyrics))

<class 'list'> 50


In [112]:
for ly in lyrics : 
    print(ly)

내 야윈 손위로 온 초대장 위에
널 데려간다는 그와 네 이름
오래전 헤어지던 날
자꾸 내 눈앞에 스쳐
혼자서도 난 서글픈데
수화기 너머로 젖은 네 목소리
그때 왜 날 보내줬냐고
첨부터 널 사랑했다면
마지막까지 붙잡아야 했다고
그 많은 사연들 다 버려둔 채로
날 떠난 너였잖아
지금 그를 사랑한다면서
왜 이제 와서 또 바보처럼 흔들려
누굴 위해 돌아봐
아픈 가슴 추억으로 달래며
네 행복을 믿었는데
차라리 날 다 지워버려
넌 그를 택한 세상에 후회하지 않도록
남들처럼 나도 멀리서 나마
축하하고 싶으니
다 기억할 만큼 모질지 못한 너
날 두 번 울리잖아
되돌리긴 너무 늦었는데
왜 이제 와서 또 바보처럼 흔들려
누굴 위해 돌아봐
아픈 가슴 추억으로 달래며
네 행복을 믿었는데
차라리 날 다 지워버려
넌 그를 택한 세상에 후회하지 않도록
남들처럼 나도 멀리서 나마
축하하고 싶으니
부디 가서 영원해 줘
돌아서는 너를 보며
난 아무 말도 할 수 없었고
슬퍼하기엔 짧았던
나의 해는 저물어 갔네
지나치는 모진 기억이
바람 따라 흩어질 때면
아무 일도 없듯이 보내주려 해
아픈 맘이 남지 않도록
안녕 멀어지는 나의 하루야
빛나지 못한 나의 별들아
차마 아껴왔던 말 이제서야
잘 지내 인사를 보낼 게
떠나가는 너를 보며
난 아무 말도 할 수 없었고
슬퍼하기엔 짧았던
나의 해는 저물어 갔네
돌이킬 순 없는 추억이
바람 따라 흩어질 때면
아무 일도 없듯이 보내주려 해
아픈 맘이 남지 않도록
안녕 멀어지는 나의 하루야
빛나지 못한 나의 별들아
차마 아껴왔던 말 이제서야
잘 지내 인사를 보낼 게
잘 지내 인사를 보낼 게
원합니다 내가 살기 위해서
내가 이렇게도 가슴이 뛰는 건
그대가 내 마음에 다녀 갔었나 봐
우리 사랑은 또 스쳐가지만
세상에서 가장 그리운 사람
내가 어느새 그대 이름을 불러
원합니다 내가 살기 위해서
그 사랑이 아파도 기다릴게 여기서
사랑이란 멀리 있어도
언젠간 만날 테니까
네가 없으면
죽을 것 같아서 살기 위해서
왜 이유 없이 눈물이 났는지
그렇게 가슴 아파야 했는지
지금 

### 📌 문장 생성기 (템플릿 적용 - mark1 )

1. 가사 데이터를 가져온다   
2. 가사를 추출    
3. 가사를 전체 다 쪼개 -> 품사 태깅 하고 이것을 새롭게 저장   
4. 품사별 사전을 만든다 (중복 없도록 함)   
5. 어순의 위치 (주어, 목적어, 서술어) 에 올 수 있는 품사를 정의 해둠    
6. 여기에 맞게 랜덤으로 사전에서 가져온다.    

In [113]:
test_lyric = lyrics[0]
len(test_lyric)

446

In [114]:
lyrics[0].replace("\n"," ")
# lyrics[0] = lyrics[0].replace("\r\n", " ").replace("\n", " ")

'내 야윈 손위로 온 초대장 위에 널 데려간다는 그와 네 이름 오래전 헤어지던 날 자꾸 내 눈앞에 스쳐 혼자서도 난 서글픈데 수화기 너머로 젖은 네 목소리 그때 왜 날 보내줬냐고 첨부터 널 사랑했다면 마지막까지 붙잡아야 했다고 그 많은 사연들 다 버려둔 채로 날 떠난 너였잖아 지금 그를 사랑한다면서 왜 이제 와서 또 바보처럼 흔들려 누굴 위해 돌아봐 아픈 가슴 추억으로 달래며 네 행복을 믿었는데 차라리 날 다 지워버려 넌 그를 택한 세상에 후회하지 않도록 남들처럼 나도 멀리서 나마 축하하고 싶으니 다 기억할 만큼 모질지 못한 너 날 두 번 울리잖아 되돌리긴 너무 늦었는데 왜 이제 와서 또 바보처럼 흔들려 누굴 위해 돌아봐 아픈 가슴 추억으로 달래며 네 행복을 믿었는데 차라리 날 다 지워버려 넌 그를 택한 세상에 후회하지 않도록 남들처럼 나도 멀리서 나마 축하하고 싶으니 부디 가서 영원해 줘'

In [115]:
from konlpy.tag import Okt, Kkma, Komoran

ret_dt = []

sentences = test_lyric.split("\n")
# sentences = lyrics[0].replace("\n", " ").strip().split(".")  # 예시: 마침표(.) 기준으로 문장 나누기
# sentences = lyrics[0].replace("\n"," ")

komoran = Komoran()

for sentence in sentences: 
    tagged_word = komoran.pos(sentence)
    for word, pos in tagged_word: 
        # ret_dt.append([sentence, word, pos])
        ret_dt.append([word, pos])

ret_dt

[['내', 'NP'],
 ['야위', 'VV'],
 ['ㄴ', 'ETM'],
 ['손위', 'NNG'],
 ['로', 'JKB'],
 ['오', 'VV'],
 ['ㄴ', 'ETM'],
 ['초대장', 'NNG'],
 ['위', 'NNG'],
 ['에', 'JKB'],
 ['너', 'NP'],
 ['ㄹ', 'JKO'],
 ['데려가', 'VV'],
 ['ㄴ다는', 'ETM'],
 ['그', 'NP'],
 ['와', 'JKB'],
 ['네', 'MM'],
 ['이름', 'NNG'],
 ['오래전', 'NNG'],
 ['헤어지', 'VV'],
 ['던', 'ETM'],
 ['날', 'NNG'],
 ['자꾸', 'MAG'],
 ['내', 'NP'],
 ['눈앞', 'NNG'],
 ['에', 'JKB'],
 ['스치', 'VV'],
 ['어', 'EC'],
 ['혼자', 'NNG'],
 ['서', 'JKB'],
 ['도', 'JX'],
 ['나', 'VV'],
 ['ㄴ', 'ETM'],
 ['서글프', 'VA'],
 ['ㄴ데', 'EC'],
 ['수화기', 'NNP'],
 ['너머', 'NNP'],
 ['로', 'JKB'],
 ['젖', 'VV'],
 ['은', 'ETM'],
 ['네', 'MM'],
 ['목소리', 'NNG'],
 ['그때', 'NNG'],
 ['왜', 'MAG'],
 ['날', 'NNG'],
 ['보내', 'VV'],
 ['어', 'EC'],
 ['주', 'VX'],
 ['었', 'EP'],
 ['냐고', 'EC'],
 ['첨', 'NNG'],
 ['부터', 'JX'],
 ['너', 'NP'],
 ['ㄹ', 'JKO'],
 ['사랑', 'NNG'],
 ['하', 'XSV'],
 ['았', 'EP'],
 ['다면', 'EC'],
 ['마지막', 'NNG'],
 ['까지', 'JX'],
 ['붙잡', 'VV'],
 ['아야', 'EC'],
 ['하', 'VV'],
 ['았', 'EP'],
 ['다고', 'EC'],
 ['그', 'MM'],
 ['많',

In [128]:
from konlpy.tag import Okt, Kkma, Komoran

# def pos_lyrics (text, tag) : 
    

# OTK 기반의 품사태깅 함수 
def pos_by_okt (text) : 
    okt = Okt()
    ret_dt = []
    sentences = text.split("\n")
    for sentence in sentences: 
        tagged_word = okt.pos(sentence)
        for word, pos in tagged_word: 
            ret_dt.append([word, pos])
    return  ret_dt

#### 여기서 중복을 어떻게 거를지 생각해야 하고 문장을 단위로 할 대안책에 대해서도 생각해야함  

# Kkma 기반의 품사태깅 함수 
def pos_by_kkma (text) : 
    kkma = Kkma()
    print("Kkma 형태소:", kkma.morphs(text))
    print("Kkma 품사 태깅:", kkma.pos(text))
    pass 

# Komoran 기반의 품사 태깅 함수 
def pos_by_komoran (text) : 
    komoran = Komoran()
    ret_dt = []
    sentences = text.split("\n")
    
    for sentence in sentences: 
        if sentence.strip() != "":
            tagged_word = komoran.pos(sentence)
            for word, pos in tagged_word: 
                ret_dt.append([word, pos])
    return  ret_dt

In [117]:
# for lyric in lyrics: 
#     sentences = lyric.split("\n")
#     for sentence in sentences: 
#         print("*"*80)
#         print(sentence)
#         preds = trained_model(sentence)[0]
#         for l, p in zip(LABELS, preds):
#             if p>0.4:
#                 print(f"{l}: {p}")

In [139]:
# 형태소 사전 만들기 ()
def make_morpheme_dict (lyrics) :
    lyric_pos = []
    for lyric in lyrics:
        lyric_pos.extend(pos_by_komoran(lyric))  

    return lyric_pos

In [ ]:
lyric_pos

In [140]:
pos_lyric = make_morpheme_dict(test_lyric)

In [146]:
pos_lyric[0]

['내', 'NP']

In [ ]:
# 품사 별 정의부 
# subject object predicate  

# 주어로 인정할 품사 목록
subject_pos = {"NNG", "NNP", "NNB", "NP", "NR"}

object_pos = {}

In [145]:
subject_dict = []

for word in pos_lyric : 
    print(word[0],word[1])
    # if word[1] in subject_pos : 
    #     subject_dict = 

내 NP
야 IC
위 NNG
ㄴ JX
손 NNG
위 NNG
로 NNG
오 VV
ㄴ ETM
초 NNB
대 NNB
장 NNP
위 NNG
에 JKB
너 NP
ㄹ JKO
데 NNB
려 EC
간 NNB
다 MAG
늘 VV
ㄴ ETM
그 MM
오 VV
아 EC
네 MM
이 MM
름 NA
오 NNP
래 NNG
전 MM
헤 VV
어 IC
지 NNB
덜 VV
ㄴ ETM
날 NNG
자 NNB
꾸 VV
내 NP
눈 NNG
앞 NNG
에 JKB
스 NA
치 VV
어 EC
혼 NNG
자 NNB
서 VV
어 EC
도 NNG
나 VV
ㄴ ETM
서 VV
어 EC
글 NNG
프 VX
ㄴ ETM
데 NNB
수 NNB
화 NNG
기 NNG
너 NP
머 NP
로 NNG
젖 NNG
은 NNP
네 MM
목 NNG
소 NNP
리 NNB
그 MM
때 NNG
왜 MAG
날 NNG
보 NNB
내 NP
주 VX
었 EP
냐 NNG
고 MM
첨 NNG
부 NNG
터 NNB
너 NP
ㄹ JKO
사 NNG
랑 NNP
하 VV
았 EP
다 MAG
면 NNG
말 VX
아 EC
지 NNB
막 MAG
까 NNP
지 NNB
붙 VV
잡 NNG
아 IC
야 IC
하 VV
았 EP
다 MAG
고 MM
그 MM
많 NA
은 NNP
사 NNG
열 VV
ㄴ ETM
들 VV
ㄹ ETM
다 MAG
버 NNP
려 EC
두 VV
ㄴ ETM
채 NNB
로 NNG
날 NNG
뜨 VV
어 EC
나 VV
ㄴ ETM
너 NP
이 VCP
었 EP
자 VV
않 VV
아 IC
지 NNB
금 NNG
그 MM
를 JKO
사 NNG
랑 NNP
한 MM
다 MAG
면 NNG
서 VV
어 EC
왜 MAG
이 MM
제 XPN
오 VV
아 EC
서 VV
어 EC
또 MAJ
바 NNB
보 NNB
처 NNG
럼 NA
흔 NA
들 VV
ㄹ ETM
려 EC
누 NNP
굴 NNG
위 NNG
하 VV
아 EC
돌 NNP
아 IC
보 VV
아 EC
아 IC
프 VX
ㄴ ETM
가 VV
아 EC
슴 NA
추 NNP
억 NR
으 NNG
로 NNG
달 NNG
래 NNG
이 VCP

### 문장의 구조 만들기 

1. **주어에 올 수 있는 형태소**  
   - **명사(N)**: 사람, 책, 나무, 서울 등  
   - **대명사(NP)**: 나, 너, 우리, 그것 등  
   - **수사(Nu)**: 하나, 둘, 셋 등 (주어 역할 가능)  
   - **조사(J)**:  
     - 주격 조사: **이/가, 께서, 에서**  
     - 보조사: **도, 만, 은/는** 등  

2. **목적어에 올 수 있는 형태소**  
   - **명사(N)**: 사과, 컴퓨터, 문제, 이야기 등  
   - **대명사(NP)**: 나, 너, 그것 등  
   - **수사(Nu)**: 하나, 둘, 셋 등 (목적어 역할 가능)  
   - **조사(J)**:  
     - 목적격 조사: **을/를**  
     - 보조사: **도, 만, 까지, 조차** 등  

3. **서술어에 올 수 있는 형태소**  
   - **동사(V)**: 가다, 먹다, 자다, 배우다 등  
   - **형용사(A)**: 예쁘다, 크다, 어렵다 등  
   - **보조 용언**:  
     - 보조 동사: **-고 있다, -아/어 보다, -게 하다** 등  
     - 보조 형용사: **-아/어 있다, -아/어 두다** 등  
   - **어미(E)**:  
     - 종결 어미: **-다, -는다, -니, -어요, -습니다** 등  
     - 연결 어미: **-고, -며, -아서/어서** 등  

In [ ]:
def process_case(value):
    match value:
        case 1:
            return "This is case 1"
        case 2:
            return "This is case 2"
        case _:
            return "This is the default case"

print(process_case(1))  # 👉 "This is case 1"
print(process_case(3))  # 👉 "This is the default case"


In [ ]:
import random

주어 = ["나는", "그녀는", "그는", "우리는", "학생들은"]
목적어 = ["책을", "영화를", "커피를", "음악을", "게임을"]
동사 = ["읽는다", "본다", "마신다", "듣는다", "한다"]

for _ in range(5):  
    문장 = f"{random.choice(주어)} {random.choice(목적어)} {random.choice(동사)}."
    print(문장)


In [59]:
# 최종 문장 만드는 함수 
def make_sentence_by_pattern (): 
    sentence = ""
    match pattern:
    # 명사 주격조사 동사  
        case 1 : 
            sentence = f"{random.choice(주어)} {random.choice(목적어)} {random.choice(동사)}."
    # 수사 주격조사 보조동사/보조형용사 종결어미 
    # 명사/대명사 주격조사 보조동사 연결어미 
    # 명사/대명사 주격조사 보조형용사 관형사형 어미 
    # 명/대/수 주격격조사 관형사 의존명사 목적격조사 형용사 
    return ""

In [134]:
# 이미 있는 문장의 형태소 조합패턴을 찾는걸 머신러닝으로 할 수 있지 않을까??

Object `있어` not found.


In [ ]:
import random

# 문장성분 별 랜덤 발생 함수 
# CASE subject object predicate  
def make_sentence_unit (element): 
    sentence = ""
    match element:
        case "subject":
            # NNG NNP NNB NP NR / JKS
            sentence = f"{random.choice(주어)} {random.choice(목적어)} {random.choice(동사)}."
            return "This is case 1"
        case "object":
            # NNG NNP NNB NP NR / JKO
            return "This is case 2"
        case "predicate":
            # VV/ VA/ VX/ VCP / VCN
            return "This is case 3"
    return sentence

In [ ]:
## 주어 + 동사 + 목적어" 패턴을 찾기 

import re

pattern = r"(\w+) (likes|eats|plays|watches) (\w+)"
text = "He likes football. She eats apples. They play games."

matches = re.findall(pattern, text)
print(matches)  # [('He', 'likes', 'football'), ('She', 'eats', 'apples'), ('They', 'play', 'games')]


In [135]:
from konlpy.tag import Komoran

# 1️. Komoran 형태소 분석기 초기화
komoran = Komoran()

# 2️. 주어로 인정할 품사 목록
subject_pos = {"NNG", "NNP", "NNB", "NP", "NR"}

# 3️. 특정 단어를 주어로 인정하는 딕셔너리
subject_dict = {"서울": "NNP", "사람": "NNG", "그": "NP", "첫째": "NR", "둘째": "NR"}

# 4️. 문장에서 주어 후보를 추출하는 함수 (품사 & 특정 단어 둘 다 고려)
def extract_subject_candidates(sentence):
    words = komoran.pos(sentence)  # 형태소 분석 수행
    
    # (1) 품사 필터링 (주어 품사만 남기기)
    filtered_subjects = [word for word, pos in words if pos in subject_pos]
    
    # (2) 특정 단어 리스트에 있는 단어 추가 (품사와 무관)
    for word, pos in words:
        if word in subject_dict:
            filtered_subjects.append(word)
    
    return list(set(filtered_subjects))  # 중복 제거 후 반환

# 5️⃣ 테스트 문장
sentence1 = "서울의 날씨가 좋다."
sentence2 = "그 사람은 첫째 아들이다."

print(extract_subject_candidates(sentence1))  # ['서울', '날씨']
print(extract_subject_candidates(sentence2))  # ['그', '사람', '첫째']


['서울', '날씨']
['아들', '사람', '첫째', '그']
